In [1]:
import os
from clip_retrieval.clip_back import load_clip_index, KnnService, ClipOptions

/home/junjie/anaconda3/envs/clip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
columns = ["url", "caption", "image_path"] #needs to be check if there are any more valid
clip_res = load_clip_index(
    clip_options=ClipOptions(
        indice_folder="/nfs/ssd14/projects/junjie/laion400m_index_tests/10ms_tune/n128_ef_256_nogpu",
        clip_model="ViT-L/14", #ViT-B/32, ViT-L/14
        enable_hdf5=False,
        enable_faiss_memory_mapping=False,
        columns_to_return=columns,
        reorder_metadata_by_ivf_index=False,
        enable_mclip_option=False,
        use_jit=True,
        use_arrow=False,
        provide_safety_model=False,
        provide_violence_detector=False,
        provide_aesthetic_embeddings=True,
        )
    )

warming up with batch size 1 on cuda


/home/junjie/anaconda3/envs/clip/lib/python3.9/site-packages/clip/clip.py:160: FutureWarning: 'torch.onnx._patch_torch._node_getitem' is deprecated in version 1.13 and will be removed in version 1.14. Please Internally use '_node_get' in symbolic_helper instead..
  if "value" in node.attributeNames() and str(node["value"]).startswith("cuda"):


done warming up in 4.72126030921936s


In [3]:
clip_res_dict = {}
clip_res_dict[0]=clip_res
knn_service_laion= KnnService(clip_resources=clip_res_dict)

In [4]:
import numpy as np

l14_weight = np.genfromtxt('/home/junjie/clip_retrieval/classifier/l14_weight.csv', delimiter=' ')
l14_bias = np.genfromtxt('/home/junjie/clip_retrieval/classifier/l14_bias.csv', delimiter=' ')

weight_np = l14_weight[0,:]
weight_p = l14_weight[1,:]
bias_np = l14_bias[0]
bias_p = l14_bias[1]

In [7]:
results = knn_service_laion.classifier_img_query(
    #cls_weight_emb=weight_np,
    #cls_bias_emb=bias_np,
    cls_weight_emb=weight_p,
    cls_bias_emb=bias_p,
    num_images=200000,
    num_result_ids=200000,
    deduplicate=True,
    use_safety_model=False,
    use_violence_detector=False,
    aesthetic_score=None,
    aesthetic_weight=None,
    threshold=0.0
)

Duration for filtering query results: 0.003235252108424902
Total duration for query with classifier: 1510.1055955938064


In [14]:
import pandas as pd
url_captions = pd.DataFrame([(e['image_path'], e['id'], e['similarity']) for e in results], columns=["image_path", "id", "similarity"])
from IPython.display import Image, display
verify_image1 = Image(filename=url_captions["image_path"][0])

In [15]:
url_captions

,image_path,id,similarity
0,/nfs/ssd3/data/laion400m/laion400m-data-09-img...,118129811,0.260879
1,/nfs/ssd2/data/laion400m/laion400m-data-02-img...,31380099,0.259428
2,/nfs/ssd8/data/laion400m/laion400m-data-20-img...,249485474,0.256691
3,/nfs/ssd3/data/laion400m/laion400m-data-09-img...,111973585,0.255502
4,/nfs/ssd4/data/laion400m/laion400m-data-18-img...,221941110,0.255139
5,/nfs/ssd6/data/laion400m/laion400m-data-04-img...,56643808,0.253858
6,/nfs/ssd9/data/laion400m/laion400m-data-28-img...,347929900,0.253393
7,/nfs/ssd9/data/laion400m/laion400m-data-30-img...,373713048,0.253164
